# Null Safety

Generally speaking, there is no real `null` in the vk language, a similar concept is `Option` type.

Option is a monad, which means that the inner value may or may not exist. 

Since such type is so common, `T?` can be used as a shorthand for `Option<T>`

For convenience, the standard library also exports:

In [ ]:
@static
let null: Any? = None();

## Monadic Raise Operator

`!` can be called on all monadic objects in vk

For the `Option<T>` type, use `!` to immediately throw a `NoneError<T>` exception, which will be automatically absorbed by the handler of type `Option<T>`.

In [ ]:
# Option a!
a.match {
    case Some(s): s
    case None   : raise NoneError()
}

For the `Result<T, E1>` type, use `!` to immediately throw an exception of `E` type, which will be automatically absorbed by the handler of type `Result<T, E2>`, where `E2: From< E1>`

In [ ]:
# Result a!
a.match {
    case Success(s): s
    case Failure(e): raise e
}

## Monadic Coalesce Operator

Suppose we have a variable of `dict: Dict<int>?`.

If you call it with `map`, `dict.map { $1.get(0) }` will get a double `Option` type like `int??`.

In general, this case should use `and_then` to combine double `Option`, `dict.ant_then { $1.get(0) }` to get `int?`.

This kind of application is also very common, so vk provides the `?.` operator.

In this case, just write `map?.get(0)`

The `?.` operator can also be applied to the pipeline composed of `Result<T, E1>, Result<T, E2>, ...`.

But the additional requirement is to satisfy `E2: From<E1>`, that is to say, the latter error type needs to be lossless convertible from the former one.

Similarly, the following call logic is also supported:

In [ ]:
a?(args)
a?[index]

## Monadic Fill Operator

Sometimes you want to use the given value when the left side is `None` or `Failure`, then you can use the `??` operator.

The sugar process is as follows:

In [ ]:
# Option a ?? b
a.match {
    case Some(s): s
    case None   : b
}
# Result a ?? b
a.match {
    case Success(s): s
    case Failure(_): b
}

## Or Pattern

Sometimes the right side is a complex progress, such as loading data from the network, or requiring user input.

At this point, you don't want to call this unless it's really need, then you can use the `||` operator.

The sugar process is as follows:

In [ ]:
# Option a || b
a.match {
    case Some(s): a
    case None   : { b }()
}

## Monadic Assignment Operator

Sometimes you want the left side to be reassigned if and only if it is null, then you can use the `?=` operator.

The sugar process is as follows:

In [ ]:
# a ?= b
a.match {
    case Some(_):
    case None   : a = b
}

In [ ]:
Note that there is no other assignment logic, such as:

In [ ]:
a ?+= 1
a ?-= 1